Installations

In [ ]:
%pip install matplotlib pandas openpyxl

Imports

In [ ]:
import subprocess
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd

Preparing folder

In [ ]:
examples_folder_path = '../examplesGenerator/'
results_folder_path = f"{examples_folder_path}results/"
n_iterations = 7
subprocess.run(f"mkdir {results_folder_path}", shell=True, stderr=subprocess.DEVNULL)
subprocess.run(f"rm {results_folder_path}*", shell=True, stderr=subprocess.DEVNULL)
subprocess.run(f"cd {examples_folder_path} && python3 examplesGenerator.py 3 {n_iterations}", shell=True, check=True)

Gathering data

In [ ]:
programs_rand_seed = 0
programs_loops_factor = 50

def get_program_compilation_time(file,opt):
    time_result : subprocess.CompletedProcess = subprocess.run(f"time clang++ -o output.o {opt} {file}",
                                                               check=True, shell=True, capture_output=True, text=True)
    if time_result.returncode != 0:
        print(f"Error in {file}")
        sys.exit(1)
    elapsed_time_str = time_result.stderr.split(' ')[2]
    time_str = elapsed_time_str.replace('elapsed', '')
    time_str_list = time_str.split(':')
    secs = float(time_str_list[-1])
    mins = 0
    if len(time_str_list) >= 2:
        mins = int(time_str_list[-2])
    hours = 0
    if len(time_str_list) >= 3:
        hours = int(time_str_list[0])
    return hours*3600 + mins*60 + secs

def get_program_running_time(file,opt):
    time_result : subprocess.CompletedProcess = subprocess.run(f"time ./output.o {programs_rand_seed} {programs_loops_factor}",
                                                               check=True, shell=True, capture_output=True, text=True)
    if time_result.returncode != 0:
        print(f"Error in {file}")
        sys.exit(1)
    elapsed_time_str = time_result.stderr.split(' ')[2]
    time_str = elapsed_time_str.replace('elapsed', '')
    time_str_list = time_str.split(':')
    secs = float(time_str_list[-1])
    mins = 0
    if len(time_str_list) >= 2:
        mins = int(time_str_list[-2])
    hours = 0
    if len(time_str_list) >= 3:
        hours = int(time_str_list[0])
    return hours*3600 + mins*60 + secs

def get_program_size(file) -> int:
    size_result : subprocess.CompletedProcess = subprocess.run(f"size output.o", shell=True, capture_output=True, text=True)
    if size_result.returncode != 0:
        print(f"Error in {file}")
        sys.exit(1)
    size_str = size_result.stdout.split('\n')[1].split('\t')
    return int(size_str[0])

def get_program_cpu_cycles() -> int:
    subprocess.run(f"perf stat -o temp.txt -e cycles ./output.o {programs_rand_seed} {programs_loops_factor}", shell=True, check=True)
    with open('temp.txt', 'r') as f:
        lines = f.readlines()
        lines = [line for line in lines if line.strip() != '']
        words = lines[2].strip().split(' ')
        cycles = int(words[0].replace('.', ''))
    return cycles

In [ ]:
recursive_calls = []
optimizations = []
sizes = []
cpu_cycles = []
running_times = []
compilation_times = []
opts = ['-O0', '-O1', '-O2', '-O3', '-Os', '-Oz', '-Ofast']
print("0.0%")
for i, file in enumerate(os.listdir(results_folder_path)):
    try:
        info_array = file.split('_')
        production_rule = int(info_array[2])
        for opt in opts:
            recursive_calls.append(production_rule)
            optimizations.append(opt)
            compilation_times.append(get_program_compilation_time(results_folder_path+file,opt))
            running_times.append(get_program_running_time(results_folder_path+file,opt))
            sizes.append(get_program_size(file))
            cpu_cycles.append(get_program_cpu_cycles())
            p100 = len(os.listdir(results_folder_path)) * len(opts)
            p = i * len(opts) + opts.index(opt) + 1
            print(f"{p/p100*100:.2f}%")
    except Exception as e:
        print(e)
        print(f"Error in {file} with optimization {opt}. Iteration {i}")
        sys.exit(1)
subprocess.run(f"rm output.o temp.txt", shell=True)

### Building tables

In [ ]:
df = pd.DataFrame({'Recursive Calls': recursive_calls, 'Optimizations': optimizations, 'Size (bytes)': sizes, 'CPU Cycles': cpu_cycles, 'Compilation Time (s)': compilation_times, 'Running Time (s)': running_times})

Getting Ratios to -O0

In [ ]:
df['CPU Cycles ratio from O0'] = 1.0
df['Compilation time ratio from O0'] = 1.0
df['Running time ratio from O0'] = 1.0

for call in df['Recursive Calls'].unique():
    o0_row = df[(df['Recursive Calls'] == call) & (df['Optimizations'] == '-O0')].iloc[0]
    
    o0_cpu_cycles = o0_row['CPU Cycles']
    o0_compilation_time = o0_row['Compilation Time (s)']
    o0_running_time = o0_row['Running Time (s)']
    
    df.loc[df['Recursive Calls'] == call, 'CPU Cycles ratio from O0'] = (o0_cpu_cycles / df.loc[df['Recursive Calls'] == call, 'CPU Cycles']).round(2)
    df.loc[df['Recursive Calls'] == call, 'Compilation time ratio from O0'] = (o0_compilation_time / df.loc[df['Recursive Calls'] == call, 'Compilation Time (s)']).round(2)
    df.loc[df['Recursive Calls'] == call, 'Running time ratio from O0'] = (o0_running_time / df.loc[df['Recursive Calls'] == call, 'Running Time (s)']).round(2)

Getting Averages

In [ ]:
avg_size = df['Size (bytes)'].mean().round(2)
avg_cpu_cycles = df['CPU Cycles'].mean().round(2)
avg_compilation_time = df['Compilation Time (s)'].mean().round(2)
avg_running_time = df['Running Time (s)'].mean().round(2)
avg_cpu_cycles_ratio = df['CPU Cycles ratio from O0'].mean().round(2)
avg_compilation_time_ratio = df['Compilation time ratio from O0'].mean().round(2)
avg_running_time_ratio = df['Running time ratio from O0'].mean().round(2)

average_row = pd.DataFrame({
    'Recursive Calls': ['Average'],
    'Optimizations': [''],
    'Size (bytes)': [avg_size],
    'CPU Cycles': [avg_cpu_cycles],
    'Compilation Time (s)': [avg_compilation_time],
    'Running Time (s)': [avg_running_time],
    'CPU Cycles ratio from O0': [avg_cpu_cycles_ratio],
    'Compilation time ratio from O0': [avg_compilation_time_ratio],
    'Running time ratio from O0': [avg_running_time_ratio]
})
df = pd.concat([df, average_row], ignore_index=True)

Sorting

In [ ]:

df.sort_values(by=['Recursive Calls', 'Optimizations'], inplace=True)

Exporting data to files

In [ ]:
subprocess.run(f"mkdir results", shell=True, stderr=subprocess.DEVNULL)
df.to_csv('results/main_table.csv', index=False)

with pd.ExcelWriter('results/tables.xlsx') as writer:
    df.to_excel(writer, sheet_name='Main Table', index=False)